In [1]:
import os
import sys
import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf
import layers
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
COLOR_CHANNELS = 3

In [3]:
MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
WEIGHTS = '../cifar10vgg_numpy.npz'
weights = np.load(WEIGHTS)

In [4]:
def load_weights(layer_name):
    layer_type = layer_name.split('_')[0]
    layer_num = int(layer_name.split('_')[1])
    W = weights["b'" + str(layer_name) + "/kernel:0'"]
    b = weights["b'" + str(layer_name) + "/bias:0'"]
    if layer_type == 'conv2d':
        beta = weights["b'batch_normalization_" + str(layer_num) + "/beta:0'"]
        gamma = weights["b'batch_normalization_" + str(layer_num) + "/gamma:0'"]
        return W, b, beta, gamma
    elif layer_type == 'dense':
        if layer_num == 1:
            beta = weights["b'batch_normalization_14/beta:0'"]
            gamma = weights["b'batch_normalization_14/gamma:0'"]
            return W, b, beta, gamma
        else:
            return W, b

In [5]:
def conv(inputs, name):
    W, b, beta, gamma = load_weights(name)
    W = tf.constant(W)
    b = tf.constant(np.reshape(b, (b.size)))
    activations = tf.nn.relu(tf.nn.conv2d(inputs, filter=W, strides=[1, 1, 1, 1], padding='SAME') + b)
    return layers.batch_norm(
        activations, 
        phase_train=tf.constant(False), 
        initial_beta=beta, 
        initial_gamma=gamma, 
        scope=name)[0]

In [6]:
def pool(inputs):
    return tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
def fc(inputs, name):
    W, b, beta, gamma = load_weights(name)
    W = tf.constant(W)
    b = tf.constant(np.reshape(b, (b.size)))
    activations = tf.nn.relu(tf.matmul(inputs, W) + b)
    return layers.batch_norm(
        activations, 
        phase_train=tf.constant(False), 
        initial_beta=beta, 
        initial_gamma=gamma, 
        scope=name)[0]

In [8]:
def softmax(inputs, name):
    W, b = load_weights(name)
    W = tf.constant(W)
    b = tf.constant(np.reshape(b, (b.size)))
    return tf.nn.softmax(tf.matmul(inputs, W) + b)

In [9]:
n_images = 1
img = tf.get_variable('img', initializer=np.zeros([n_images, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS], 
                                                  dtype=np.float32))

conv1_1 = conv(img, 'conv2d_1')
conv1_2 = conv(conv1_1, 'conv2d_2')
pool1 = pool(conv1_2)

conv2_1 = conv(pool1, 'conv2d_3')
conv2_2 = conv(conv2_1, 'conv2d_4')
pool2 = pool(conv2_2)

conv3_1 = conv(pool2, 'conv2d_5')
conv3_2 = conv(conv3_1, 'conv2d_6')
conv3_3 = conv(conv3_2, 'conv2d_7')
pool3 = pool(conv3_3)

conv4_1 = conv(pool3, 'conv2d_8')
conv4_2 = conv(conv4_1, 'conv2d_9')
conv4_3 = conv(conv4_2, 'conv2d_10')
pool4 = pool(conv4_3)

conv5_1 = conv(pool4, 'conv2d_11')
conv5_2 = conv(conv5_1, 'conv2d_12')
conv5_3 = conv(conv5_2, 'conv2d_13')
pool5 = pool(conv5_3)

flattened = tf.reshape(pool5, [-1, 512])
fc6 = fc(flattened, 'dense_1')
pred = softmax(fc6, 'dense_2')

In [10]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        if sys.version[0] == '3':
            dict = pickle.load(fo, encoding='bytes')
        elif sys.version[0] == '2':
            dict = pickle.load(fo)
    return dict

In [11]:
def normalize(X_test):
    mean = np.mean(X_test,axis=(0, 1, 2, 3))
    std = np.std(X_test, axis=(0, 1, 2, 3))
    X_test = (X_test - mean)/(std + 1e-7)
    return X_test

In [12]:
cifar10 = unpickle('cifar-10-data')
permutation = np.random.permutation(10000)
data = np.array(cifar10[b'data'])[permutation]
labels = np.array(cifar10[b'labels'])[permutation]
x_test = normalize(data.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8"))[:n_images]
y_test = labels[:n_images]

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(img.assign(x_test))

y = tf.placeholder(tf.int64, [None,], name='y')
correct = tf.equal(tf.argmax(pred, axis=1), y)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
print(sess.run([accuracy], feed_dict={y:y_test}))

[0.0]


In [13]:
def generate_noise_image():
    return np.random.uniform(-20, 20, (1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)).astype('float32')

In [14]:
def content_loss_func(sess, layer):
    def _content_loss(l, x):
        return tf.nn.l2_loss(x - l)
    return _content_loss(sess.run(layer), layer)

In [15]:
CONTENT_IMAGE = 'images/cornell-campus.jpg'
OUTPUT_DIR = 'reconstructions/'
CONTENT_LAYER = conv4_2

In [16]:
def load_image(path):
    image = scipy.misc.imread(path)
    image = np.reshape(image, ((1,) + image.shape)) 
    image = image - MEAN_VALUES                     
    return image

def save_image(path, image):
    image = image + MEAN_VALUES                     
    scipy.misc.imsave(path, image)

In [17]:
content_image = load_image(CONTENT_IMAGE)
input_image = generate_noise_image()

In [18]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

sess.run(img.assign(content_image))
content_loss = content_loss_func(sess, conv4_2)

optimizer = tf.train.AdamOptimizer(2.0)
train_step = optimizer.minimize(content_loss)

ValueError: Dimension 1 in both shapes must be equal, but are 32 and 256 for 'Assign_1' (op: 'Assign') with input shapes: [1,32,32,3], [1,256,256,3].

In [ ]:
ITERATIONS = 1000

In [ ]:
for it in range(ITERATIONS):
    sess.run(train_step)
    if it%100 == 0:
        reconstruction = sess.run(img)
        print('Iteration %d' % (it))
        print('loss: ', sess.run(content_loss))

        if not os.path.exists(OUTPUT_DIR):
            os.mkdir(OUTPUT_DIR)

        filename = 'output/%d.png' % (it)
        save_image(filename, mixed_image)